## Introduction


All data used in this project is provided by AQUASTAT, free of charge. The database can be accessed directly at: [https://www.fao.org/aquastat/statistics/query/index.html]. 

The complete dataset consists of 193 variables classified as:

1 - **Geography and population**:

    - Land use
    - Population
    - Economy, development and food security


2 -  **Water resources**:

    - Precipitation
    - Internal renewable water resources
    - External renewable water resources 
    - Total renewable water resources
    - Exploitable water resources and dam capacity


3 - **Water use**:

    - Water withdrawal by sector
    - Water withdrawal by source
    - Wastewater
    - Pressure on water resources

4 - **Irrigation and drainage development**:

    - Area under agricultural water management
    - Area equipped for irrigation by source of water
    - Power irrigated area
    - Irrigated crop area and cropping intensity
    - Irrigated crop yield
    - Drainage

5 - **Environment and health**:

    - Environment
    - Access to improved drinking water source
    - Health


For this projected, I've download each subcategory data as CSV (Flat) file, selecting all countries and time period (1950 to 2020). Since the datasource has a limit of rows to be displayed, some subcategories were split into two or more files.

Some subcategories weren't available for download or visualization because of a bug in the FAOSTAT website. For those subcategories, the files were download from an alternative website [https://data.harvestportal.org/pt_PT/dataset/fao-aquastat]. Because of that, some subcategories may have not updated values. The downloaded files:
 - Water withdrawal by sector
 - Water withdrawal by source
 - Wastewater
 - Area under agricultural water management
 - Irrigated crop area and cropping intensity



All files were saved at the 'base' folder and named following the same structure: 
"aquastat" + subcategory name separated by "_ " + part number (if subcategory was split)




#### Data First Sight

In [3]:
import pandas as pd
from os import listdir
import re
import os.path

In [6]:
data_folder = globals()['_dh'][0] +'\\dataset\\'
original_csv_folder = globals()['_dh'][0] +'\\dataset\\base\\'
aquastat_files = listdir(original_csv_folder)

All CSV (Flat) files follow the same structure:

- Columns with the properties:
    - Area (country name)
    - Area Id (ISO-3166 code for each country)
    - Variable Name (measured variable)
    - Year (year the variable was measured)
    - Value (value of the measured variable)
    - Symbol (remarks regarding the value)
    - Md (code for associated metadata)
- Rows with metadata [if there's any]
- Footer with (c) FAO's info


In [62]:
# Direct import trial
pd.read_csv(original_csv_folder + aquastat_files[0])

,Area,Area Id,Variable Name,Variable Id,Year,Value,Symbol,Md
Afghanistan,4.0,Total population with access to safe drinking-...,4114.0,1992.0,21.3,X,NaN,6603.0
Afghanistan,4.0,Total population with access to safe drinking-...,4114.0,1997.0,25.0,X,NaN,6603.0
Afghanistan,4.0,Total population with access to safe drinking-...,4114.0,2002.0,33.8,X,NaN,6603.0
Afghanistan,4.0,Total population with access to safe drinking-...,4114.0,2007.0,42.6,X,NaN,6603.0
Afghanistan,4.0,Total population with access to safe drinking-...,4114.0,2012.0,51.6,X,NaN,6603.0
...,...,...,...,...,...,...,...,...
-- General,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-- Accuracy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
(c) FAO of the UN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
The information contained in AQUASTAT is provided free of charge to all users. Recommended citation:<br/>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## CSV modifications

Because of the metadata and footer rows, we can't import csvs directly into a dataframe. To remove the last rows, I could use the **'skipfooter = x'** parameter when calling pd_read.csv(), but unlike the default footer, the number of rows containing metadata changes, and because of that I would have to pass **a specific value of x for each file**. Instead of exploring each file to get that x value just to cut the final rows, I'll create **two new file types** during the exploration: [1] **csv** without footer or meta; [2] **txt** file with metadata referring to the original file. The txt file will be used to 'translate' the codes that are given in the Md column.

In [63]:
# Output folders
fixed_folder = globals()['_dh'][0] +'\\dataset\\fixed_csvs\\'
meta_folder = globals()['_dh'][0] +'\\dataset\\metadata\\'

#### Creating new CSV and txt

In [ ]:
for aqua_file in aquastat_files:

    original = original_csv_folder + aqua_file  # source
    output = fixed_folder + aqua_file
    metadata = meta_folder + aqua_file + '.metadata.txt'
    nf = 0

    with open(original, "r") as rf, open(output, "w") as wf:
        for line in rf:
            if '-- Coverage' in line:  # Footer begins
                break
            elif 'Metadata:' in line:  # Metadata begins
                nf = 1  # Stop writing CSV
                mf = open(metadata, "w")

            if nf == 0:  # Writes CSV
                wf.write(line)
            else:  # Writes txt
                mf.write(line)

        if nf == 1:
            mf.close()  # If there's a txt, close it


In [ ]:
# First version of new csv
pd.read_csv(fixed_folder + aquastat_files[0])

#### Alterando novos CSVs

All footer and metadata rows have been dropped, but the values ​​in each column still don't match the Header of the file.
Checking the documentation on the Faostat website, I found that the Md column can include one or more codes, which are **comma separated**. This causes a new column to be automatically created, but the Header doesn't take this into account. To solve this, I'll pass a new Header, splitting the Md column into **Md1** and **Md2**:
- Md1 contains codes related to the metadata that was at the end of the original file
- Md2 contains codes that are not listed in the original file*


Thinking about the dash that I'll make later, some other considerations were made:

- Md1 can be an interesting part of hoovertext in graphics. To make the text more user-friendly, I'm going to translate the codes for the associated explanations, which are in the txt files.
- Some information that is displayed on the FAOSTAT website is not included in the files when they are downloaded, such as the units of each measured variable and the time interval associated with each measurement. As these variables can be interesting, I will add them to the dataset.
- To create a map, it is necessary to pass the 3-letter ISO-3166 code for each location as an argument, so this information is relevant to be included.

Thus, 3 new columns will be created:
- **ISO-alpha**: column with the 3 letters corresponding to the ISO-3166 code of each country, so that it can be passed as an argument when creating maps.
- **Units**: column with the measurement units of each variable
- **Period**: column with the range of years associated with each measurement



To create the units column, a csv in Table format (including all variables for a single country in a single time interval) was downloaded from the AQUASTAT website and will be used as a reference.
- The Period column will be based on the timeline displayed on the AQUASTAT download page.



###### *So far, I haven't been able to find any information about what those codes were. The most likely explanation is that they refer to the data source, but since I'm not sure, I won't use it.

In [ ]:
# Helper functions

from apps.helpers.ISO_dicts import *

def get_units(column):

    """
    Recebe a coluna que incluí o nome da variável e sua unidade escritas em conjunto, 
    no formato "Nome da variável + (observação opcional) + (unidade)", e retorna uma lista
    com os valores "Nome da variável + (observação opcional)" e outra lista com "unidade", 
    com índices correspondentes 
    """
    
    column = column.tolist()
    m_var = [re.search(r'^.+?(?=\(([^)]+)\)[^)]*\Z)', x)[0].strip() for x in column] # get full var name
    m_unit = [re.search(r'\(([^)]+)\)[^)]*\Z', x).group(1) for x in column] # get ONLY unit of measurement
    m_unit = [x.replace("^9 ", u"\u2079 ") for x in m_unit]
    m_unit = [x.replace("10", u"\u00B7 10") for x in m_unit]
    m_unit = [x.replace("m3", u"m\u00B3") for x in m_unit]
    m_unit = [x.replace("m2", u"m\u00B2") for x in m_unit]
    return m_var, m_unit

def match_time(x):  

    # return period (YYYY, YYYY) in which an x year is placed

    years_mean = range(1960,2023,5) # from AQUASTAT website
    years_period = [(i-2, i+2) for i in years_mean]

    for period in years_period:
        if x >= period[0] and x <= period[1]:
            return str(period)


def fix_md1(column, meta_list): 

    # list whit medatada explanations "[XXX]"
    output = [] 
    
    for x in column.tolist():
        if x == '-':
            output.append(x)
        else:
            x = int(re.search(r'\d+',  x)[0])  
            output.append(re.search(r'\d+.(.*?)\n',  meta_list[x]).group(1)) # get explanation from txt
    return output


def fix_units(column, prop_list, unit_list): 
    
    
    output = []

    for x in column.tolist():
        index = [i for i, j in enumerate(prop_list) if j == x] 
        output.append(unit_list[index[0]]) # gets only the first unit of measurement (sometimes there's a second version with worst formatting)
    return output





In [ ]:
# Using a reference CSV
df_units = pd.read_csv(data_folder+'aquastat_units.csv',  header=0, names=[0,1,2,3,4,5]) 
df_units.drop(df_units.tail(7).index, inplace=True) # Remove footer and empty row

prop_list, unit_list = get_units(df_units[1])

# I've later noticed that variableids are quite useful, so I'll store them now
variableids = []


In [ ]:
# Modifying CSVs

for aqua_file in aquastat_files:

    fixed = fixed_folder + aqua_file
    column_names = ["Area","AreaId","VariableName","VariableId","Year","Value","Symbol","Md1","Md2"] # new header
    aqua_df = pd.read_csv(fixed, header=0, names=column_names)
    
    #
    aqua_df.drop(aqua_df.tail(1).index, inplace=True) # last row is NaN, always 
    aqua_df['Value'].fillna(value=0, inplace=True)
    aqua_df.fillna(value="-", inplace=True) 
    

    aqua_df['Value'] = pd.to_numeric(aqua_df['Value'])
    
    # replace metadata only if it exists
    metadata = meta_folder + aqua_file + '.metadata.txt'
    if os.path.isfile(metadata): 
        with open(metadata, "r") as f:
            m = f.readlines()
            aqua_df['Md1'] = fix_md1(aqua_df['Md1'], m)

    ## create new columns     
    
    aqua_df['Units'] = fix_units(aqua_df['VariableName'], prop_list, unit_list)  
    aqua_df['Period'] = [match_time(x) for x in aqua_df['Year'].tolist()]
    aqua_df['ISOCode'] = [num_to_alpha3(x) for x in aqua_df['AreaId'].tolist()]
    aqua_df.to_csv(fixed, index=False) # save the final csv version


    variableids.append(aqua_df.VariableId.unique())
    
    

In [ ]:
# Check new files
aqua_df

#### Creating file with Definitions and Calculation Formula

A (very) useful piece of information for users is the **definition** of each variable is calculated and how it was calculated. Although this information is not available in csvs, it is available in pop-ups on the AQUASTAT website, always following the same URL pattern: "https://www.fao.org/aquastat/statistics/popups/itemDefn.html?id=**x**"where **x** is the Variable Id.

As the URLS follow the same pattern, they can be accessed iteratively quite easily*! So, I'm going to create a separate csv file, "def_and_calc.csv", with the respective information for each VariableId

* I won'tt use libraries like BeautifulSoup or Selenium because the site information wasn't explicitly organized inside tags. I would have to search through strings anyway, so I'll use RegEX directly.


In [ ]:
# Fix dimension of original ids list
def flatten(L):
    for item in L:
        try:
            yield from flatten(item)
        except TypeError:
            yield item

variableids = list(flatten(variableids))


In [ ]:
import requests

definitions, calculations = [], []

for id in variableids:

    URL = "https://www.fao.org/aquastat/statistics/popups/itemDefn.html?id=" + str(int(id))
    page = requests.get(URL)

    # some definitions have line breaks, to use regex I'll need to change what . search!
    d = re.compile(r'<strong>Definition</strong><br/>(.*?)<br/><br/>', re.DOTALL)    
    d_base =  d.findall(page.text)[0]

    # Check if there really is a definition; if not, the title of the next property will be returned (Calculation Criteria or Unit)
    if re.search(r'Calculation Criteria</strong>', d_base) or re.search(r'<strong>Unit</strong>', d_base):
        d_base = '-'

    definitions.append(d_base)

    # Search if Calculation exist 
    calc_base = re.search(r'Calculation Criteria</strong><br/>(.*?)<br/><br/><strong>Unit</strong>', page.text)

    if calc_base:
        calc_base = re.sub(r'\[<a+(.*?)\">', "", calc_base.group(1)) # remove hyperlinks
        calculations.append(re.sub("</a>]", "", calc_base))
    else:
        calculations.append('-')



In [ ]:
# Export CSV
def_and_calc = pd.DataFrame({'VariableId': variableids, 'Definition': definitions, 'Calculation': calculations})
def_and_calc.to_csv(data_folder+'def_and_calc.csv', index=False)
# def_and_calc

##  Dash Use

All files are now ready to be used in the dash!

I've decided to make a function that pulls all files from a folder and returns two dictionaries:
- one with all dataframes in the folder, where the key is the formatted name* of each file**
- one with all properties on each file, where the key is also the formatted name of each file

 This is **not** necessary (nor necessarily more efficient), but I chose to access it that way in the dashboard. 
 

 ###### *All csv files are named as: "aquastat_" + [subcategory name separated by "_" ] + ".csv". The formatted version is the file name separated by " " and with each first letter capitalized.

  ###### **I've created two new subcategories only for the dashboard. They don't have a csv file, as they are just a manipulation of an alredy existing one.



In [36]:
from apps.helpers.aqua import get_aqua_files

dfs, aqua_var = get_aqua_files(aquastat_files, fixed_folder)

# # Subcategories
# len(aqua_var.keys()) #ou dfs.keys()

# # Dict of variables
# aqua_var

# # Dict of dataframes
# dfs

23